In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, glob

# Try both spellings (because your Drive folder name may differ)
CANDIDATES = [
    "/content/drive/MyDrive/DataSets/Assaignment_Videos",
    "/content/drive/MyDrive/DataSets/Assaignment_Videos",
]
VIDEOS_DIR = None
for c in CANDIDATES:
    if os.path.exists(c):
        VIDEOS_DIR = c
        break

if VIDEOS_DIR is None:
    raise FileNotFoundError("❌ Could not find videos folder. Check your Drive path.")

PROJECT_ROOT = "/content/drive/MyDrive/SC549_Assignment03_PlayerTracking"
BASE_OUT     = os.path.join(PROJECT_ROOT, "outputs")

OUT_YOLO      = os.path.join(BASE_OUT, "yolo")
OUT_TRACKING  = os.path.join(BASE_OUT, "tracking")
OUT_KEYPOINTS = os.path.join(BASE_OUT, "keypoints")

for p in [OUT_YOLO, OUT_TRACKING, OUT_KEYPOINTS]:
    os.makedirs(p, exist_ok=True)

print("✅ VIDEOS_DIR:", VIDEOS_DIR)
print("✅ Output root:", BASE_OUT)
print("✅ Files:", os.listdir(VIDEOS_DIR))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ VIDEOS_DIR: /content/drive/MyDrive/DataSets/Assaignment_Videos
✅ Output root: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs
✅ Files: ['Vedio_1.mp4', 'Vedio_2.mp4', 'vedio_3.mp4', 'edited_vedio1.mp4', 'edited_vedio2.mp4', 'Edited_vedio3.mp4', 'edited_video4.mp4', 'edited_video5.mp4']


In [ ]:
video_files = sorted(glob.glob(os.path.join(VIDEOS_DIR, "*.mp4")))
if len(video_files) == 0:
    raise FileNotFoundError("❌ No .mp4 files found in your videos folder!")

print("✅ Found videos:", len(video_files))
for v in video_files:
    print(" -", os.path.basename(v))


✅ Found videos: 8
 - Edited_vedio3.mp4
 - Vedio_1.mp4
 - Vedio_2.mp4
 - edited_vedio1.mp4
 - edited_vedio2.mp4
 - edited_video4.mp4
 - edited_video5.mp4
 - vedio_3.mp4


In [ ]:
!pip -q install ultralytics
!apt-get -y -q install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.2 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [ ]:
import os, glob, subprocess, torch
from ultralytics import YOLO

# Use GPU if available
DEVICE = 0 if torch.cuda.is_available() else "cpu"
print("✅ Using device:", "GPU" if DEVICE == 0 else "CPU")

det_model   = YOLO("yolov8n.pt")
track_model = YOLO("yolov8n.pt")
pose_model  = YOLO("yolov8n-pose.pt")

RUNS_ROOT = "/content/runs_sc549"

def avi_to_mp4(avi_path, mp4_path):
    os.makedirs(os.path.dirname(mp4_path), exist_ok=True)
    cmd = ["ffmpeg", "-y", "-i", avi_path, "-vcodec", "libx264", "-pix_fmt", "yuv420p", mp4_path]
    subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)

def get_latest_avi(folder):
    avis = glob.glob(os.path.join(folder, "*.avi"))
    if not avis:
        return None
    return max(avis, key=os.path.getmtime)

✅ Using device: CPU


In [ ]:
for vp in video_files:
    name = os.path.splitext(os.path.basename(vp))[0]
    print("\n🟦 DETECT:", name)

    det_model.predict(
        source=vp,
        save=True,
        project=os.path.join(RUNS_ROOT, "detect"),
        name=name,
        exist_ok=True,
        verbose=False,
        device=DEVICE
    )

    det_run = os.path.join(RUNS_ROOT, "detect", name)
    det_avi = get_latest_avi(det_run)

    if det_avi:
        det_mp4 = os.path.join(OUT_YOLO, f"{name}_detect.mp4")
        avi_to_mp4(det_avi, det_mp4)
        print("✅ Detect MP4 saved:", det_mp4)
    else:
        print("❌ Detect AVI not found:", det_run)


🟦 DETECT: Edited_vedio3
WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to /content/runs_sc549/detect/Edited_vedio3
✅ Detect MP4 saved: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs/yolo/Edited_vedio3_detect.mp4

🟦 DETECT: Vedio_1
WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example

In [ ]:
for vp in video_files:
    name = os.path.splitext(os.path.basename(vp))[0]
    print("\n🟩 TRACK:", name)

    # stream=True returns a generator -> we must consume it
    for _ in track_model.track(
        source=vp,
        save=True,
        persist=True,
        stream=True,
        project=os.path.join(RUNS_ROOT, "tracking"),
        name=name,
        exist_ok=True,
        verbose=False,
        device=DEVICE
    ):
        pass

    track_run = os.path.join(RUNS_ROOT, "tracking", name)
    track_avi = get_latest_avi(track_run)

    if track_avi:
        track_mp4 = os.path.join(OUT_TRACKING, f"{name}_tracking.mp4")
        avi_to_mp4(track_avi, track_mp4)
        print("✅ Tracking MP4 saved:", track_mp4)
    else:
        print("❌ Tracking AVI not found:", track_run)


🟩 TRACK: Edited_vedio3
Results saved to /content/runs_sc549/tracking/Edited_vedio3
✅ Tracking MP4 saved: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs/tracking/Edited_vedio3_tracking.mp4

🟩 TRACK: Vedio_1
Results saved to /content/runs_sc549/tracking/Vedio_1
✅ Tracking MP4 saved: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs/tracking/Vedio_1_tracking.mp4

🟩 TRACK: Vedio_2
Results saved to /content/runs_sc549/tracking/Vedio_2
✅ Tracking MP4 saved: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs/tracking/Vedio_2_tracking.mp4

🟩 TRACK: edited_vedio1
Results saved to /content/runs_sc549/tracking/edited_vedio1
✅ Tracking MP4 saved: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs/tracking/edited_vedio1_tracking.mp4

🟩 TRACK: edited_vedio2
Results saved to /content/runs_sc549/tracking/edited_vedio2
✅ Tracking MP4 saved: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs/tracking/edited_vedio2_tracking.mp4



In [ ]:
for vp in video_files:
    name = os.path.splitext(os.path.basename(vp))[0]
    print("\n🟥 KEYPOINTS:", name)

    pose_model.predict(
        source=vp,
        save=True,
        project=os.path.join(RUNS_ROOT, "pose"),
        name=name,
        exist_ok=True,
        verbose=False,
        device=DEVICE
    )

    pose_run = os.path.join(RUNS_ROOT, "pose", name)
    pose_avi = get_latest_avi(pose_run)

    if pose_avi:
        pose_mp4 = os.path.join(OUT_KEYPOINTS, f"{name}_keypoints.mp4")
        avi_to_mp4(pose_avi, pose_mp4)
        print("✅ Keypoints MP4 saved:", pose_mp4)
    else:
        print("❌ Pose AVI not found:", pose_run)

print("\n🎉 DONE: Detect + Track + Keypoints for all videos!")


🟥 KEYPOINTS: Edited_vedio3
WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to /content/runs_sc549/pose/Edited_vedio3
✅ Keypoints MP4 saved: /content/drive/MyDrive/SC549_Assignment03_PlayerTracking/outputs/keypoints/Edited_vedio3_keypoints.mp4

🟥 KEYPOINTS: Vedio_1
WARNING ⚠️ 
Inference results will accumulate in RAM unless `stream=True` is passed, which can cause out-of-memory errors for large
sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for